In [18]:
from dotenv import dotenv_values
import os

config = dotenv_values(".env.secret")  # load sensitive variables
print(config.keys())

for c, v in config.items():
    os.environ[c] = v

odict_keys(['WANDB_API_KEY', 'WANDB_NAME', 'WANDB_NOTES', 'WANDB_NOTEBOOK_NAME'])


In [19]:
import wandb

wandb_key = os.environ["WANDB_API_KEY"]
wandb.login(key=wandb_key)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [20]:
import torch

torch.cuda.is_available()

True

In [21]:
import torch

torch.cuda.device_count()

1

In [22]:
import torch

torch.cuda.get_device_name()

'NVIDIA GeForce GTX 1650'

In [23]:
import os

max(1, os.cpu_count() -1)

11

In [24]:
import os
import wandb
import torch
from lightning import Trainer
from pytorch_lightning.loggers import WandbLogger
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from src.model import NeuralNetwork
from src.data import HousePricingDataModule
from src.util import fake_name

project_name = os.environ["WANDB_NAME"]

patience = 50
max_epochs = 500
accelerator = "gpu" if torch.cuda.is_available() else "cpu"

# Setting up the training configuration
config = {
    "accelerator": accelerator,
    "max_epochs": max_epochs,
    "patience": patience
}

test_results = dict()

for _, batch_size in enumerate([256, 512]):

    for i in range(9):

        run_name = fake_name()

        # Preparing data to be used
        data_module = HousePricingDataModule(batch_size=batch_size)
        data_module.prepare_data()

        # Requesting to data module the number of different features that are in the data
        in_features = data_module.in_features()

        config["batch_size"] = batch_size
        config["in_features"] = in_features

        wandb.init(
            job_type="training",
            name=run_name,
            project=project_name,
            config=config,
        )

        # Defining the model to be training
        model = NeuralNetwork(input_size=wandb.config["in_features"])

        checkpoint_name = f"{wandb.run.name}-{wandb.run.id}"

        # Defining the logger instance the lighning will use as default logging
        logger = WandbLogger(
            log_model="all",
            checkpoint_name=checkpoint_name
        )

        # Define how the model registry work
        checkpoint_callback = ModelCheckpoint(
            every_n_epochs=1,
            monitor="val_loss",
            mode = "min"
        )

        # Defining early stop configuration
        early_stopping_callback = EarlyStopping(
            monitor="val_loss",
            mode="min",
            verbose=True,
            patience=wandb.config["patience"]
        )

        # Defines the training instance
        trainer = Trainer(
            accelerator=wandb.config["accelerator"],
            max_epochs=wandb.config["max_epochs"],
            logger=logger,
            log_every_n_steps=1,
            callbacks=[checkpoint_callback, early_stopping_callback]
        )

        trainer.fit(model, datamodule=data_module)

        test_result = trainer.test(model, datamodule=data_module)
        test_results[checkpoint_name] = test_result

        wandb.finish()

[INFO]: Preprocessing training dataframe...
[INFO]: Dropping columns with full of NA or Identifiers. Current dataframe shape: (1460, 81)
[INFO]: Dropped columns with full of NA or Identifiers. Current dataframe shape: (1460, 74)
[INFO]: Dropping categories types with few ocurrence. Current dataframe shape: (1460, 74)
[INFO]: Few ocurrences removed. Current dataframe shape: (1443, 74)
[INFO]: Dropping columns which contains just one type of category. Current dataframe shape: (1443, 74)
[INFO]: Columns with just one type of categroy dropped. Current dataframe shape: (1443, 73)
[INFO]: Preprocessing predict dataframe...
[INFO]: Dropping columns with full of NA or Identifiers. Current dataframe shape: (1459, 80)
[INFO]: Dropped columns with full of NA or Identifiers. Current dataframe shape: (1459, 73)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


[INFO]: Input size: 244
[INFO]: Preprocessing training dataframe...
[INFO]: Dropping columns with full of NA or Identifiers. Current dataframe shape: (1460, 81)
[INFO]: Dropped columns with full of NA or Identifiers. Current dataframe shape: (1460, 74)
[INFO]: Dropping categories types with few ocurrence. Current dataframe shape: (1460, 74)
[INFO]: Few ocurrences removed. Current dataframe shape: (1443, 74)
[INFO]: Dropping columns which contains just one type of category. Current dataframe shape: (1443, 74)
[INFO]: Columns with just one type of categroy dropped. Current dataframe shape: (1443, 73)
[INFO]: Preprocessing predict dataframe...
[INFO]: Dropping columns with full of NA or Identifiers. Current dataframe shape: (1459, 80)
[INFO]: Dropped columns with full of NA or Identifiers. Current dataframe shape: (1459, 73)


/root/miniconda3/envs/pricehousing/lib/python3.9/site-packages/pytorch_lightning/loggers/wandb.py:391: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type       | Params
------------------------------------
0 | net  | Sequential | 215 K 
------------------------------------
215 K     Trainable params
0         Non-trainable params
215 K     Total params
0.864     Total estimated model params size (MB)


[INFO]: Setting up fit dataset/s
[INFO]: Train dataset size: 981
[INFO]: Validation dataset size: 173
Sanity Checking: |          | 0/? [00:00<?, ?it/s][INFO]: Validation dataloader size: 1
                                                                            

...

Monitored metric val_loss did not improve in the last 50 records. Best score: 5817078784.000. Signaling Trainer to stop.


Epoch 51: 100%|██████████| 2/2 [00:00<00:00, 15.11it/s, v_num=af1k]
[INFO]: Preprocessing training dataframe...
[INFO]: Dropping columns with full of NA or Identifiers. Current dataframe shape: (1460, 81)
[INFO]: Dropped columns with full of NA or Identifiers. Current dataframe shape: (1460, 74)
[INFO]: Dropping categories types with few ocurrence. Current dataframe shape: (1460, 74)
[INFO]: Few ocurrences removed. Current dataframe shape: (1443, 74)
[INFO]: Dropping columns which contains just one type of category. Current dataframe shape: (1443, 74)
[INFO]: Columns with just one type of categroy dropped. Current dataframe shape: (1443, 73)
[INFO]: Preprocessing predict dataframe...
[INFO]: Dropping columns with full of NA or Identifiers. Current dataframe shape: (1459, 80)
[INFO]: Dropped columns with full of NA or Identifiers. Current dataframe shape: (1459, 73)


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/root/miniconda3/envs/pricehousing/lib/python3.9/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


[INFO]: Setting up test dataset/s
[INFO]: Test dataloader size: 1
Testing DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 109.74it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_loss              5250480640.0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
test_loss,▁
train_loss,█▂▅██▇█▇██▇▇█▇▇▇▆▅▄▃▂▁▂▂▂▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁
trainer/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
val_loss,▂▁▇██████████▇▇▇▆▄▃▂▁▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,52
test_loss,5250480640.0
train_loss,5459275264.0
trainer/global_step,104
val_loss,6803472384.0


In [25]:
test_results

{'Jennifer_1Z54Q60B-7pyxyr9j': [{'test_loss': 656872704.0}],
 'Lauren_I60OXE4F-kujs20js': [{'test_loss': 679620800.0}],
 'Mary_WD1U1DKL-0ku76atb': [{'test_loss': 5248507904.0}],
 'Diana_F631Q4QW-c6t7xxk6': [{'test_loss': 696892864.0}],
 'Melissa_6SL9D4QD-6j68n3gp': [{'test_loss': 36675817472.0}],
 'Kristy_WHBCJ7MM-rmboixur': [{'test_loss': 667574400.0}],
 'Caleb_K86AVDNZ-ggi6sr6y': [{'test_loss': 663124288.0}],
 'Dawn_T5QCSZQX-932vsjo5': [{'test_loss': 655516672.0}],
 'Andrew_8Q4O5LJ0-vskvlukg': [{'test_loss': 656359360.0}],
 'Robert_U8MLZTSV-a5052b3b': [{'test_loss': 649291776.0}],
 'Anne_S5TYFMO5-qcjhf28a': [{'test_loss': 5450159616.0}],
 'Kimberly_PN9PBRQM-uwb5bw1b': [{'test_loss': 18807910400.0}],
 'Briana_5KJREAU0-3s0bbhzn': [{'test_loss': 5274674688.0}],
 'Justin_IFPBF97F-u7rf76ng': [{'test_loss': 36679151616.0}],
 'Dana_6SY7MFXS-t02nqub8': [{'test_loss': 36679364608.0}],
 'James_6G45TE6G-q364k7pm': [{'test_loss': 5516247552.0}],
 'Joshua_L84TM1E8-8axi6zl2': [{'test_loss': 527445

In [32]:
sss = {name: value[0]["test_loss"] for name, value in test_results.items()}
print(f"Best model in test dataset: {min(sss, key=sss.get)}")
print(f"Worst model in test dataset: {max(sss, key=sss.get)}")

Best model in test dataset: Robert_U8MLZTSV-a5052b3b
Worst model in test dataset: Dana_6SY7MFXS-t02nqub8
